In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
plt.style.use("ggplot")
%matplotlib inline

from transform import get_X_y
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# import tensorflow modules. tensorflow was run on a docker container

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import Precision, Recall

# tf.config.threading.set_intra_op_parallelism_threads(12)

In [6]:
## import sklearn modules

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

In [7]:
orbit = pd.read_csv('../data/samples_2_bodies_3_dim_1_m_com.csv')

In [8]:
orbit

,sim_id,m_1,m_2,rx_1_0,ry_1_0,rz_1_0,rx_2_0,ry_2_0,rz_2_0,vx_1_0,...,rz_1_1000,rx_2_1000,ry_2_1000,rz_2_1000,vx_1_1000,vy_1_1000,vz_1_1000,vx_2_1000,vy_2_1000,vz_2_1000
0,0.0,1.0,1.0,0.072812,0.339681,0.234807,-0.072812,-0.339681,-0.234807,0.032668,...,0.245414,-0.062839,-0.156803,-0.245414,-0.104210,-0.721673,-0.262195,0.104210,0.721673,0.262195
1,0.0,1.0,1.0,0.054407,0.105241,0.222055,-0.054407,-0.105241,-0.222055,-0.151459,...,0.001457,-0.017101,-0.250963,-0.001457,0.167632,0.763737,0.546329,-0.167632,-0.763737,-0.546329
2,0.0,1.0,1.0,0.015466,-0.034834,0.083430,-0.015466,0.034834,-0.083430,-0.452646,...,0.080874,-0.039404,-0.331130,-0.080874,0.128510,0.343728,0.494653,-0.128510,-0.343728,-0.494653
3,0.0,1.0,1.0,0.060140,0.364899,0.167492,-0.060140,-0.364899,-0.167492,0.083064,...,0.261195,-0.074577,-0.281935,-0.261195,-0.013481,-0.439943,0.074785,0.013481,0.439943,-0.074785
4,0.0,1.0,1.0,0.017181,0.251330,0.001720,-0.017181,-0.251330,-0.001720,0.167503,...,0.181902,-0.063202,-0.364724,-0.181902,0.074315,-0.030131,0.357845,-0.074315,0.030131,-0.357845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,1999.0,1.0,1.0,-0.056611,0.192364,-0.247187,0.056611,-0.192364,0.247187,-0.214536,...,-0.125396,0.077384,-0.004295,0.125396,0.130841,-1.165302,1.163385,-0.130841,1.165302,-1.163385
199996,1999.0,1.0,1.0,-0.062381,-0.046865,-0.059740,0.062381,0.046865,0.059740,0.604474,...,0.126458,-0.132333,-0.099748,-0.126458,0.632941,1.154271,0.048515,-0.632941,-1.154271,-0.048515
199997,1999.0,1.0,1.0,0.250455,0.441298,0.031888,-0.250455,-0.441298,-0.031888,0.071420,...,-0.021397,-0.256480,-0.517709,0.021397,-0.002030,0.345303,-0.286874,0.002030,-0.345303,0.286874
199998,1999.0,1.0,1.0,0.117538,0.560847,-0.275648,-0.117538,-0.560847,0.275648,-0.211689,...,-0.294678,-0.076410,-0.505170,0.294678,-0.231047,-0.354711,-0.072457,0.231047,0.354711,0.072457


In [9]:
steps = 10
X, y = get_X_y(orbit, steps, False, True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
# create and compile neural network

n_samples, n_feats = X_train.shape
_, n_outputs = y_train.shape

model = Sequential()

hidden_units = n_feats
epochs=100
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8

scale = 10


hidden_layer = Dense(units=hidden_units,
                input_dim=n_feats,
                kernel_initializer='uniform',
                activation='relu')

hidden_layer2 = Dense(units=hidden_units*scale,
                input_dim=hidden_units,
                kernel_initializer='uniform',
                activation='relu')

hidden_layer3 = Dense(units=hidden_units*scale,
                input_dim=hidden_units*scale,
                kernel_initializer='uniform',
                activation='relu')

hidden_layer4 = Dense(units=n_outputs,
                input_dim=hidden_units*scale,
                kernel_initializer='uniform',
                activation='relu')

output_layer = Dense(units=n_outputs,
                input_dim=hidden_units,
                kernel_initializer='uniform',
                activation='linear')

model.add(hidden_layer)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer2)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer4)
# model.add(Dropout(rate=0.1))
model.add(output_layer)
model.add(Dropout(rate=0.1))

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
# sgd = SGD(lr=learning_rate)
model.compile(loss='mean_squared_error', 
              optimizer=sgd)

In [11]:
history = model.fit(X_train, y_train, 
          epochs=epochs, 
          batch_size=5000, 
          verbose=2, 
          validation_split=0.2)

Train on 120000 samples, validate on 30000 samples
Epoch 1/100
120000/120000 - 1s - loss: 5.0919 - val_loss: 0.6448
Epoch 2/100
120000/120000 - 0s - loss: 1.5528 - val_loss: 0.4618
Epoch 3/100
120000/120000 - 0s - loss: 1.5055 - val_loss: 0.4584
Epoch 4/100
120000/120000 - 0s - loss: 1.5316 - val_loss: 0.4584
Epoch 5/100
120000/120000 - 0s - loss: 1.5225 - val_loss: 0.4585
Epoch 6/100
120000/120000 - 0s - loss: 1.5113 - val_loss: 0.4584
Epoch 7/100
120000/120000 - 0s - loss: 1.5193 - val_loss: 0.4584
Epoch 8/100
120000/120000 - 0s - loss: 1.5288 - val_loss: 0.4581
Epoch 9/100
120000/120000 - 0s - loss: 1.5257 - val_loss: 0.4583
Epoch 10/100
120000/120000 - 0s - loss: 1.5193 - val_loss: 0.4584
Epoch 11/100
120000/120000 - 0s - loss: 1.5267 - val_loss: 0.4584
Epoch 12/100
120000/120000 - 0s - loss: 1.5093 - val_loss: 0.4583
Epoch 13/100
120000/120000 - 0s - loss: 1.5447 - val_loss: 0.4583
Epoch 14/100
120000/120000 - 0s - loss: 1.5172 - val_loss: 0.4583
Epoch 15/100
120000/120000 - 0s - 

In [12]:
# fig, ax = plt.subplots(figsize=(12,8))
# ax.set_title('Loss')
# ax.plot(np.sqrt(history.history['loss']), label='train')
# ax.plot(np.sqrt(history.history['val_loss']), label='test')
# ax.legend();
y_pred = model.predict(X_test)
error_10 = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
print(steps, error_10)

10 [0.46569351 0.50055616 0.49530384 0.83313271 0.83056881 0.81280555]


In [13]:
y_test[0], y_pred[0]

(array([-0.30473428,  1.43991498, -0.27456941, -0.16877983, -0.00682965,
         0.29557322]),
 array([-0.28559414,  1.2847446 , -0.23912589, -0.01190691,  0.06963748,
        -0.0063195 ], dtype=float32))

In [14]:
steps = 100
X, y = get_X_y(orbit, steps, False, True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
history = model.fit(X_train, y_train, 
          epochs=epochs, 
          batch_size=5000, 
          verbose=2, 
          validation_split=0.2)

Train on 120000 samples, validate on 30000 samples
Epoch 1/100
120000/120000 - 0s - loss: 1.6182 - val_loss: 0.5231
Epoch 2/100
120000/120000 - 0s - loss: 1.6168 - val_loss: 0.5171
Epoch 3/100
120000/120000 - 0s - loss: 1.6124 - val_loss: 0.5153
Epoch 4/100
120000/120000 - 0s - loss: 1.6218 - val_loss: 0.5140
Epoch 5/100
120000/120000 - 0s - loss: 1.6449 - val_loss: 0.5136
Epoch 6/100
120000/120000 - 0s - loss: 1.6112 - val_loss: 0.5153
Epoch 7/100
120000/120000 - 0s - loss: 1.6509 - val_loss: 0.5139
Epoch 8/100
120000/120000 - 0s - loss: 1.6044 - val_loss: 0.5144
Epoch 9/100
120000/120000 - 0s - loss: 1.6353 - val_loss: 0.5125
Epoch 10/100
120000/120000 - 0s - loss: 1.6461 - val_loss: 0.5153
Epoch 11/100
120000/120000 - 0s - loss: 1.6376 - val_loss: 0.5136
Epoch 12/100
120000/120000 - 0s - loss: 1.6438 - val_loss: 0.5141
Epoch 13/100
120000/120000 - 0s - loss: 1.5967 - val_loss: 0.5133
Epoch 14/100
120000/120000 - 0s - loss: 1.6395 - val_loss: 0.5144
Epoch 15/100
120000/120000 - 0s - 

In [16]:
# fig, ax = plt.subplots(figsize=(12,8))
# ax.set_title('Loss')
# ax.plot(np.sqrt(history.history['loss']), label='train')
# ax.plot(np.sqrt(history.history['val_loss']), label='test')
# ax.legend();
y_pred = model.predict(X_test)
error_100 = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
print(steps, error_100)

100 [0.55043457 0.58400466 0.5762359  0.82846864 0.84025464 0.82381291]


In [17]:
y_test[0], y_pred[0]

(array([ 3.07684949,  1.44293991, -0.66873498,  0.44161339, -0.26461157,
         0.27543053]),
 array([ 2.5195193 ,  1.384956  , -0.5703058 ,  0.12889645,  0.07012776,
        -0.02369317], dtype=float32))

In [18]:
steps = 1000
X, y = get_X_y(orbit, steps, False, True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
history = model.fit(X_train, y_train, 
          epochs=epochs, 
          batch_size=5000, 
          verbose=2, 
          validation_split=0.2)

Train on 120000 samples, validate on 30000 samples
Epoch 1/100
120000/120000 - 0s - loss: 3.5394 - val_loss: 1.5640
Epoch 2/100
120000/120000 - 0s - loss: 3.3160 - val_loss: 1.5721
Epoch 3/100
120000/120000 - 0s - loss: 3.3825 - val_loss: 1.5551
Epoch 4/100
120000/120000 - 0s - loss: 3.3788 - val_loss: 1.5495
Epoch 5/100
120000/120000 - 0s - loss: 3.3677 - val_loss: 1.5507
Epoch 6/100
120000/120000 - 0s - loss: 3.2844 - val_loss: 1.5443
Epoch 7/100
120000/120000 - 0s - loss: 3.3606 - val_loss: 1.5372
Epoch 8/100
120000/120000 - 0s - loss: 3.3605 - val_loss: 1.5401
Epoch 9/100
120000/120000 - 0s - loss: 3.3176 - val_loss: 1.5483
Epoch 10/100
120000/120000 - 0s - loss: 3.3331 - val_loss: 1.5207
Epoch 11/100
120000/120000 - 0s - loss: 3.3359 - val_loss: 1.5382
Epoch 12/100
120000/120000 - 0s - loss: 3.2692 - val_loss: 1.5429
Epoch 13/100
120000/120000 - 0s - loss: 3.3312 - val_loss: 1.5320
Epoch 14/100
120000/120000 - 0s - loss: 3.3225 - val_loss: 1.5242
Epoch 15/100
120000/120000 - 0s - 

In [20]:
# fig, ax = plt.subplots(figsize=(12,8))
# ax.set_title('Loss')
# ax.plot(np.sqrt(history.history['loss']), label='train')
# ax.plot(np.sqrt(history.history['val_loss']), label='test')
# ax.legend();
y_pred = model.predict(X_test)
error_1000 = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
print(steps, error_1000)

1000 [1.51218904 1.59188272 1.57587394 0.81430325 0.81265348 0.80331397]


In [21]:
y_test[0], y_pred[0]

(array([-12.25550051, -27.8060288 ,   5.85832625,  -0.39903096,
         -0.98290801,   0.23287413]),
 array([-11.78412   , -25.626331  ,   5.334454  ,  -0.45046738,
         -1.009466  ,   0.23156069], dtype=float32))

In [22]:
print(np.mean(error_10[:3]), np.mean(error_10[3:]))
print(np.mean(error_100[:3]), np.mean(error_100[3:]))
print(np.mean(error_1000[:3]), np.mean(error_1000[3:]))

0.48718450329195334 0.8255023558412633
0.5702250433965922 0.8308453961088554
1.5599819009454432 0.8100902337759422
